# Eventual assertions for FluentAssertions



The extension allows to loop FluentAssertions checks for a specified period of time. The extension relies heavily on the [Assertion Scopes](https://fluentassertions.com/introduction#assertion-scopes) feature and is an assertion scope itself. Any failed assertion among those wrapped with the extension will cause another attempt. The last try is always done out of the Assertion Scope (under the parent scope)

A particular goal when designing the extension was to provide the experience similar to given by Assertion Scopes, that is, no additional nesting would be needed. This is elegantly achievable in F\#, but a little bit of a challenge for C\#

## EventualAssertions in C\#

In [14]:
#r "nuget: mazharenko.FluentAssertions.Eventual, *"

Installed Packages mazharenko.FluentAssertions.Eventual, 3.0.0

In [15]:
using mazharenko.FluentAssertions.Eventual;
using FluentAssertions;
using FluentAssertions.Extensions;

var attempts = EventualAssertions.Attempts(4.Seconds(), 400.Milliseconds());

`attempts` now is of a type that implements `IEnumerable` returning `IEnumerator` wrapping an assertion scope. On each new element requested the assertion scope is checked and the specified delay is waited. Here, assertions are supposed to be placed in the body of the loop over `attempts`. Elements of `attempts` can provide some meta information about the attempt.

In [16]:
#!time
var inFuture = DateTime.Now.AddSeconds(3);

foreach (var attempt in attempts)
{
    Console.WriteLine(attempt.Number);
    DateTime.Now.Should().BeAfter(inFuture);
}

8
9


Wall time: 3237.1159ms

In [17]:
foreach (var _ in EventualAssertions.Attempts(400, 50))
    (1+1).Should().Be(3);

Error: FluentAssertions.Execution.AssertionFailedException: Expected value to be 3, but found 2.
   at FluentAssertions.Execution.FallbackTestFramework.Throw(String message)
   at FluentAssertions.Execution.TestFrameworkProvider.Throw(String message)
   at FluentAssertions.Execution.DefaultAssertionStrategy.HandleFailure(String message)
   at FluentAssertions.Execution.AssertionScope.FailWith(Func`1 failReasonFunc)
   at FluentAssertions.Numeric.NumericAssertions`2.Be(T expected, String because, Object[] becauseArgs)
   at Submission#10.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

EventualAssertions can be nested in other assertion scopes.

In [18]:
using FluentAssertions.Execution;

using (new AssertionScope())
{
    foreach (var _ in EventualAssertions.Attempts(400, 50))
    {
        (1+1).Should().Be(3);
        break;
    }
    foreach (var _ in EventualAssertions.Attempts(400, 50))
    {
        (2*2).Should().Be(5);
        break;
    }
}

Error: FluentAssertions.Execution.AssertionFailedException: Expected value to be 3, but found 2.
Expected value to be 5, but found 4.

   at FluentAssertions.Execution.FallbackTestFramework.Throw(String message)
   at FluentAssertions.Execution.TestFrameworkProvider.Throw(String message)
   at FluentAssertions.Execution.CollectingAssertionStrategy.ThrowIfAny(IDictionary`2 context)
   at Submission#11.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

Likewise, other assertion scopes can be nested in EventualAssertions

In [19]:
foreach (var _ in EventualAssertions.Attempts(400, 30))
{
    using (new AssertionScope()) 
    {
        (1+1).Should().Be(3);
        (2*2).Should().Be(5);
    }
}

Error: FluentAssertions.Execution.AssertionFailedException: Expected value to be 3, but found 2.
Expected value to be 5, but found 4.

   at FluentAssertions.Execution.FallbackTestFramework.Throw(String message)
   at FluentAssertions.Execution.TestFrameworkProvider.Throw(String message)
   at FluentAssertions.Execution.CollectingAssertionStrategy.ThrowIfAny(IDictionary`2 context)
   at Submission#12.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

Asynchronous methods are also available. Those would use `Task.Delay` instead of `Thread.Sleep`.

In [20]:
#!time
var inFuture = DateTime.Now.AddSeconds(3);
await foreach(var _ in EventualAssertions.AttemptsAsync(4.Seconds(), 400.Milliseconds()))
{
    DateTime.Now.Should().BeAfter(inFuture);
}

Wall time: 3226.7602ms

## EventualAssertions in F\#

The F\# implementation provides a special looping computation expression builder.

In [21]:
#r "nuget: mazharenko.FluentAssertions.Eventual.FSharp, *"

open mazharenko.FluentAssertions.Eventual.EventualAssertions

Installed Packages mazharenko.FluentAssertions.Eventual.FSharp, 3.0.0

In [22]:
#!time

open FluentAssertions

let inFuture = DateTime.Now.AddSeconds(float 3)
(eventually (TimeSpan.FromSeconds(4)) (TimeSpan.FromMilliseconds(40))) {
    DateTime.Now.Should().BeAfter(inFuture, null, null) |> ignore
}

Wall time: 3031.7765ms

In [23]:
(eventuallyMs 400 40) {
    (1+1).Should().Be(3, null, null) |> ignore
}

Error: FluentAssertions.Execution.AssertionFailedException: Expected value to be 3, but found 2.
   at FluentAssertions.Execution.FallbackTestFramework.Throw(String message)
   at FluentAssertions.Execution.TestFrameworkProvider.Throw(String message)
   at FluentAssertions.Execution.DefaultAssertionStrategy.HandleFailure(String message)
   at FluentAssertions.Execution.AssertionScope.FailWith(Func`1 failReasonFunc)
   at FluentAssertions.Numeric.NumericAssertions`2.Be(T expected, String because, Object[] becauseArgs)
   at FSI_0013.it@2-9.Invoke(Unit unitVar)
   at mazharenko.FluentAssertions.Eventual.EventualAssertions.EventualAssertionsBuilder.Run(FSharpFunc`2 f)
   at <StartupCode$FSI_0013>.$FSI_0013.main@()
   at System.RuntimeMethodHandle.InvokeMethod(Object target, Void** arguments, Signature sig, Boolean isConstructor)
   at System.Reflection.MethodInvoker.Invoke(Object obj, IntPtr* args, BindingFlags invokeAttr)

EventualAssertions builder can be nested in other assertion scopes.

In [24]:
open FluentAssertions.Execution

using (new AssertionScope()) (fun _ -> 
    (eventuallyMs 0 0) {
      (1+1).Should().Be(3, null, null) |> ignore
    }
    (eventuallyMs 0 0) {
      (2*2).Should().Be(5, null, null) |> ignore
    }
)

Error: FluentAssertions.Execution.AssertionFailedException: Expected value to be 3, but found 2.
Expected value to be 5, but found 4.

   at FluentAssertions.Execution.FallbackTestFramework.Throw(String message)
   at FluentAssertions.Execution.TestFrameworkProvider.Throw(String message)
   at FluentAssertions.Execution.CollectingAssertionStrategy.ThrowIfAny(IDictionary`2 context)
   at Microsoft.FSharp.Core.Operators.Using[T,TResult](T resource, FSharpFunc`2 action) in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 5160
   at <StartupCode$FSI_0014>.$FSI_0014.main@()
   at System.RuntimeMethodHandle.InvokeMethod(Object target, Void** arguments, Signature sig, Boolean isConstructor)
   at System.Reflection.MethodInvoker.Invoke(Object obj, IntPtr* args, BindingFlags invokeAttr)

Likewise, other assertion scopes can be nested in EventualAssertions

In [25]:
open FluentAssertions.Execution

(eventuallyMs 400 40) {
    using (new AssertionScope()) (fun _ -> 
        (1+1).Should().Be(3, null, null) |> ignore
        (2*2).Should().Be(5, null, null) |> ignore
    )
}

Error: FluentAssertions.Execution.AssertionFailedException: Expected value to be 3, but found 2.
Expected value to be 5, but found 4.

   at FluentAssertions.Execution.FallbackTestFramework.Throw(String message)
   at FluentAssertions.Execution.TestFrameworkProvider.Throw(String message)
   at FluentAssertions.Execution.CollectingAssertionStrategy.ThrowIfAny(IDictionary`2 context)
   at Microsoft.FSharp.Core.Operators.Using[T,TResult](T resource, FSharpFunc`2 action) in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 5160
   at FSI_0015.it@4-13.Invoke(Unit unitVar)
   at mazharenko.FluentAssertions.Eventual.EventualAssertions.EventualAssertionsBuilder.Run(FSharpFunc`2 f)
   at <StartupCode$FSI_0015>.$FSI_0015.main@()
   at System.RuntimeMethodHandle.InvokeMethod(Object target, Void** arguments, Signature sig, Boolean isConstructor)
   at System.Reflection.MethodInvoker.Invoke(Object obj, IntPtr* args, BindingFlags invokeAttr)

Asynchronous methods are also available. Those would use `Async.Sleep` instead of `Thread.Sleep`.

In [26]:
#!time

let inFuture = DateTime.Now.AddSeconds(3)
(eventuallyAsyncMs 4000 400) {
    DateTime.Now.Should().BeAfter(inFuture, null, null) |> ignore
} |> Async.RunSynchronously

Wall time: 3236.747ms